In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sidetable
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
pd.options.display.max_columns = None

In [13]:
df = pd.read_csv("datos/bank_additional_sin_outliers.csv", index_col = 0)

# Pair Programming Limpieza V

### Hipótesis

1. La edad, el trabajo, el estado civil, la educación, la situación de deuda y la forma de contacto pueden influir en la probabilidad de que un cliente acepte la oferta.

2. El número de veces que se ha contactado a un cliente en el pasado (campo campaign), el número de días que han pasado desde el último contacto (campo pdays), y el resultado de la campaña anterior (campo poutcome) pueden afectar la respuesta del cliente a una nueva oferta.

3. Las variables económicas (tales como el índice de precios al consumidor (cons.price.idx), la tasa de variación del empleo (emp.var.rate), etc.) pueden influir en la probabilidad de que un cliente acepte la oferta.

4. Los clientes que ya tienen una hipoteca (housing) o un préstamo (loan) pueden ser menos propensos a aceptar una nueva oferta, ya que podrían estar limitados financieramente.

### Tareas

1. Es el momento de eliminar los nulos:

- Reemplazad los valores nulos del resto de las columnas categóricas por la moda, usando el método SimpleImputer. 

💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [22]:
df.isnull().sum()

age                  0
job                330
marital             80
education         1730
default           8596
housing            990
loan               990
contact              0
duration             0
campaign             0
pdays                0
previous             0
poutcome             0
emp_var_rate         0
cons_price_idx       0
cons_conf_idx        0
euribor3m            0
nr_employed          0
y                    0
month_day_week       0
month                0
day                  0
dtype: int64

In [14]:
df_cat = df.select_dtypes(include='object')
df_cat

,job,marital,education,default,housing,loan,contact,poutcome,y,month_day_week,month,day
0,housemaid,married,basic 4y,no,no,no,telephone,NONEXISTENT,no,"['may', 'mon']",may,mon
1,services,married,high school,NaN,no,no,telephone,NONEXISTENT,no,"['may', 'mon']",may,mon
2,services,married,high school,no,si,no,telephone,NONEXISTENT,no,"['may', 'mon']",may,mon
3,administration,married,basic 6y,no,no,no,telephone,NONEXISTENT,no,"['may', 'mon']",may,mon
4,services,married,high school,no,no,si,telephone,NONEXISTENT,no,"['may', 'mon']",may,mon
...,...,...,...,...,...,...,...,...,...,...,...,...
41183,retired,married,professional course,no,si,no,cellular,NONEXISTENT,yes,"['nov', 'fri']",nov,fri
41184,blue-collar,married,professional course,no,no,no,cellular,NONEXISTENT,no,"['nov', 'fri']",nov,fri
41185,retired,married,university degree,no,si,no,cellular,NONEXISTENT,no,"['nov', 'fri']",nov,fri
41186,technician,married,professional course,no,no,no,cellular,NONEXISTENT,yes,"['nov', 'fri']",nov,fri


In [15]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [16]:
imputer = imputer.fit(df_cat)

In [17]:
imputer

SimpleImputer(strategy='most_frequent')

In [43]:
df_cat = pd.DataFrame(imputer.transform(df_cat), columns = df_cat.columns)
df_cat

,job,marital,education,default,housing,loan,contact,poutcome,y,month_day_week,month,day
0,housemaid,married,basic 4y,no,no,no,telephone,NONEXISTENT,no,"['may', 'mon']",may,mon
1,services,married,high school,no,no,no,telephone,NONEXISTENT,no,"['may', 'mon']",may,mon
2,services,married,high school,no,si,no,telephone,NONEXISTENT,no,"['may', 'mon']",may,mon
3,administration,married,basic 6y,no,no,no,telephone,NONEXISTENT,no,"['may', 'mon']",may,mon
4,services,married,high school,no,no,si,telephone,NONEXISTENT,no,"['may', 'mon']",may,mon
...,...,...,...,...,...,...,...,...,...,...,...,...
41171,retired,married,professional course,no,si,no,cellular,NONEXISTENT,yes,"['nov', 'fri']",nov,fri
41172,blue-collar,married,professional course,no,no,no,cellular,NONEXISTENT,no,"['nov', 'fri']",nov,fri
41173,retired,married,university degree,no,si,no,cellular,NONEXISTENT,no,"['nov', 'fri']",nov,fri
41174,technician,married,professional course,no,no,no,cellular,NONEXISTENT,yes,"['nov', 'fri']",nov,fri


In [19]:
df_cat.isnull().sum() #se han eliminado los nulos. 

job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
poutcome          0
y                 0
month_day_week    0
month             0
day               0
dtype: int64

In [45]:
df.drop(columns = df_cat.columns, inplace=True)
df.sample()

,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,age
20587,180,3,999,0,1.4,93.444,-36.1,4.966,5228.1,36.0


In [46]:
df = pd.concat([df, df_cat], axis = 1, join = 'inner')

In [47]:
df.sample()

,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,age,job,marital,education,default,housing,loan,contact,poutcome,y,month_day_week,month,day
5362,76,4,999,0,1.1,93.994,-36.4,4.857,5191.0,33.0,services,single,high school,no,no,no,telephone,NONEXISTENT,yes,"['may', 'fri']",may,fri


- Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [20]:
df_num = df.select_dtypes(include= np.number)
df_num


,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0
...,...,...,...,...,...,...,...,...,...,...
41183,38,334,1,999,0,-1.1,94.767,-50.8,1.028,4963.6
41184,46,383,1,999,0,-1.1,94.767,-50.8,1.028,4963.6
41185,56,189,2,999,0,-1.1,94.767,-50.8,1.028,4963.6
41186,44,442,1,999,0,-1.1,94.767,-50.8,1.028,4963.6


In [24]:
imputer2 = IterativeImputer()
imputer2

IterativeImputer()

In [25]:
imputer2.fit(df_num)

IterativeImputer()

In [30]:
df_num_trans = pd.DataFrame(imputer2.transform(df_num), columns = df_num.columns)
df_num_trans

,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
0,56.0,261.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
1,57.0,149.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
2,37.0,226.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
3,40.0,151.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
4,56.0,307.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
...,...,...,...,...,...,...,...,...,...,...
41171,38.0,334.0,1.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6
41172,46.0,383.0,1.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6
41173,56.0,189.0,2.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6
41174,44.0,442.0,1.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6


- Reemplazad los valores nulos de la columna age por la media de la edad usando el método KNN Imputer.

In [31]:
imputerKNN = KNNImputer(n_neighbors=5)
imputerKNN.fit(df_num)

KNNImputer()

In [33]:
num_knn= imputerKNN.transform(df_num)

In [35]:
df_knn_imputer = pd.DataFrame(num_knn, columns = df_num.columns)
df_knn_imputer

,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
0,56.0,261.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
1,57.0,149.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
2,37.0,226.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
3,40.0,151.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
4,56.0,307.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
...,...,...,...,...,...,...,...,...,...,...
41171,38.0,334.0,1.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6
41172,46.0,383.0,1.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6
41173,56.0,189.0,2.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6
41174,44.0,442.0,1.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6


In [37]:
df.drop(columns= ['age'], inplace=True)

In [38]:
df['age'] = df_knn_imputer['age']
df.head()

,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,month_day_week,month,day,age
0,housemaid,married,basic 4y,no,no,no,telephone,261,1,999,0,NONEXISTENT,1.1,93.994,-36.4,4.857,5191.0,no,"['may', 'mon']",may,mon,56.0
1,services,married,high school,NaN,no,no,telephone,149,1,999,0,NONEXISTENT,1.1,93.994,-36.4,4.857,5191.0,no,"['may', 'mon']",may,mon,57.0
2,services,married,high school,no,si,no,telephone,226,1,999,0,NONEXISTENT,1.1,93.994,-36.4,4.857,5191.0,no,"['may', 'mon']",may,mon,37.0
3,administration,married,basic 6y,no,no,no,telephone,151,1,999,0,NONEXISTENT,1.1,93.994,-36.4,4.857,5191.0,no,"['may', 'mon']",may,mon,40.0
4,services,married,high school,no,no,si,telephone,307,1,999,0,NONEXISTENT,1.1,93.994,-36.4,4.857,5191.0,no,"['may', 'mon']",may,mon,56.0


- Reemplazad los valores nulos de la columna age por la media de la edad usando el método Simple Imputer.

In [39]:
imputer_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

In [40]:
imputer_mean = imputer_mean.fit(df_num[['age']])

In [41]:
df_num['age'] = (imputer_mean.transform(df_num[['age']]))
df_num

,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
0,56.0,261.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
1,57.0,149.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
2,37.0,226.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
3,40.0,151.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
4,56.0,307.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
...,...,...,...,...,...,...,...,...,...,...
41171,38.0,334.0,1.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6
41172,46.0,383.0,1.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6
41173,56.0,189.0,2.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6
41174,44.0,442.0,1.0,999.0,0.0,-1.1,94.767,-50.8,1.028,4963.6


- ¿Podríais explicar qué diferencia hay entre estos últimos tres ejercicios?

Se han utilizado métodos diferentes para la sustitución de nulos. Con SimpleImputer hemos sustituido los valores de variables categóricas (ejercicio 1) y de las variables numéricas (ejercicio 4)

Para los dos ejercicios restantes se han utilizado métodos exclusivos para el tratamiento de nulos en variables numéricas, con KNNImputer e IterativeImputer. 

2. Guardad el csv.

In [50]:
df.to_csv('datos/bank_sin_nulos_imputado.csv')